# Runbook for converting [Maxim models](https://github.com/google-research/maxim) into Tensorflow.js-compatible models.

This Notebook demonstrates how to convert Maxim models into Tensorflow.js equivalents. It aims to produce smaller models that are capable of being run in the browser.

[This Notebook uses the Tensorflow port of the models](https://github.com/sayakpaul/maxim-tf).

Start by installing dependencies, and cloning the relevant repository.

In [1]:
!pip uninstall tensorflowjs -y
!pip install scikit-image numpy tensorflow tensorflow_hub matplotlib jax flax Pillow tensorflowjs
# huggingface_hub pandas tensorflowjs
!cd node && npm install && cd ..

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 37.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 45.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 43.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      S

In [2]:
# Copy the forked version, avoid any future conflicts
!git clone https://github.com/thekevinscott/maxim-tf.git cloned-code/maxim_tf

fatal: destination path 'cloned-code/maxim_tf' already exists and is not an empty directory.


In [3]:
%load_ext autoreload
%autoreload 2

import pathlib
import os
import numpy as np

## Model Definitions & Constants

[Here are all the MAXIM trained model checkpoints](https://github.com/google-research/maxim#results-and-pre-trained-models). We choose the models with the best performance (aside from Dehazing, for which we choose both indoor and outdoor, and Enhancement, for which we use both datasets).

We also have model-specific images we'll use to run inference against.

In [4]:
models_and_images = [
    # (Task, Dataset, Sample Image)
    ('Dehazing', 'SOTS-Indoor', 'https://github.com/google-research/maxim/raw/main/maxim/images/Dehazing/input/1444_10.png'),
    ('Dehazing', 'SOTS-Outdoor','https://github.com/google-research/maxim/raw/main/maxim/images/Dehazing/input/0003_0.8_0.2.png'),
    ('Denoising', 'SIDD', 'https://github.com/google-research/maxim/raw/main/maxim/images/Denoising/input/0003_30.png'),
    ('Deblurring', 'GoPro', 'https://github.com/google-research/maxim/raw/main/maxim/images/Deblurring/input/1fromGOPR0950.png'),
    ('Deraining', 'Rain13k', 'https://github.com/google-research/maxim/raw/main/maxim/images/Deraining/input/15.png'),
    ('Enhancement', 'LOL', 'https://github.com/google-research/maxim/raw/main/maxim/images/Enhancement/input/a4541-DSC_0040-2.png'),
    ('Enhancement', 'FiveK', 'https://github.com/google-research/maxim/raw/main/maxim/images/Enhancement/input/a4541-DSC_0040-2.png'),
]

And choose output folders:

In [5]:
PYTHON_MODEL_OUTPUT_FOLDER = pathlib.Path('./models/tf/python')
TFJS_MODEL_OUTPUT_FOLDER = pathlib.Path('./models/tf/tfjs')
IMAGES_OUTPUT_FOLDER = pathlib.Path('./images')
SHOULD_EVALUATE_MODELS = True # Whether we should evaluate the models after creating them
QUANTIZATION_SETTINGS = 'uint8' # Set to '' for an uncompressed version. Can also choose float16 or uint16
INPUT_RESOLUTION = 256 # If you want to specify a different input resolution from the base 256px

## Create the Models

In [ ]:
import subprocess
from PIL import Image
from evaluate import evaluate_models

for task, dataset, sample_image in models_and_images[:1]:
    try:
        q_folder = (QUANTIZATION_SETTINGS if QUANTIZATION_SETTINGS else 'uncompressed')
        res_folder = (f'{INPUT_RESOLUTION}' if INPUT_RESOLUTION else 'none')
        tfjs_output = TFJS_MODEL_OUTPUT_FOLDER / task / dataset / q_folder / res_folder
        python_output = PYTHON_MODEL_OUTPUT_FOLDER / task / dataset / res_folder
        cmd = [
            'python3',
            'create_tf_models.py',
            '--task',
            task,
            '--dataset',
            dataset,
            '--python_output',
            str(python_output),
            '--tfjs_output',
            str(tfjs_output),
            *([
                '--quantization_settings',
                f'"{QUANTIZATION_SETTINGS}"',
            ] if QUANTIZATION_SETTINGS else []),
            ######
            # optionally, set a different input resolution size.
            # input resolution must be fixed. By default, it is 256.
            # A smaller input resolution will result in smaller patches,
            # longer inference time, but less UI blocking in the browser.
            ######            
            *([
                '--input_resolution',
                f'{INPUT_RESOLUTION}',
            ] if INPUT_RESOLUTION else []),
        ]
        print(' '.join(cmd))
        subprocess.run(cmd, cwd='./')
        print(f'Created Python and Tensorflow.js models for task "{task}" and dataset "{dataset}"')
        print(f'Evaluating Python and Tensorflow.js models for task "{task}" and dataset "{dataset}"')
        evaluate_models(str(python_output), str(tfjs_output), sample_image, input_resolution=INPUT_RESOLUTION)

    except Exception as e:
        print(f'**** There was a problem with task {task} and dataset {dataset}')
        print(e)
        print('------------------------------------------------------------------------------------------')
